# Imports

In [1]:
# Basics
import pandas as pd  # for data manipulation and analysis
import numpy as np   # for numerical operations

# Sklearn
from sklearn.pipeline import Pipeline # for building preprocessing pipelines
from sklearn.impute import SimpleImputer # for imputing missing values
from sklearn.preprocessing import StandardScaler, OneHotEncoder # for feature scaling and encoding
from sklearn.model_selection import train_test_split # for splitting data into training and validation sets
from sklearn.compose import ColumnTransformer # for applying different pipelines to different columns

# Other
import joblib # for saving our final preprocessor object
import os # to get working directory

# Load Data

First thing is to load in the data.

In [2]:
# Define the file path to the training data for clarity and easy modification
file_path = "C:/Users/barbe/OneDrive/URI DS Program/566 Advanced Topics in Machine Learning/Project/neural-credit-risk-explainer/data/train.csv"

# Load the training dataset from the specified path into a pandas DataFrame
df_train = pd.read_csv(file_path, low_memory=False)

We set `low_memory=False` since a column was found to have mixed types.
- This tells pandas to read the file in chunks and infer types more accurately
- This usually resolves the warning without changing anything else.

# Initial Inspection

Next, we do our initial inspection of the dataset.

In [3]:
df_train.head()  # display the first 5 rows of the dataset

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


Note:
- ID, Customer_ID, and Name are unique identifiers or personal information, which isn't useful for modeling. These will likely be dropped in the cleaning step. Similarly for SSN.
- We can see that one row has -500 for age, which clearly can't be. We'll need to clean this column and possiblt filter out extreme values.
- Some rows are shown to have missing values. Well inspect this more and determine a course of action.
- Credit_Mix has a values of `_`, which could be an error.
- Payment_of_Min_Amount is "Yes"/"No" — perfect for binary encoding.
- Payment_Behaviour has long string patterns — may be too granular for a baseline model.

Next, we'll use the .info() method to get a technical summary of the dataset. This is crucial for understanding the data types of each column, identifying the number of non-null entries, and seeing the memory usage. It gives us a roadmap for which columns need type conversion or have missing data.

In [4]:
df_train.info()  # shows column names, non-null counts, and data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

Note:
- The output shows we have 100,000 entries and 28 columns.
- We have 28 columns comprised of object, int64, and float64 dtypes.
- 20 variables are objects, though many look like they should be numeric (Age, Annual_Income, etc). This confirms they will need to be converted.
- We can also see that several columns have fewer than 100,000 non-null entries, indicating the presence of missing values that we must handle.

We'll need to convert object-type accordingly. Missing values can be imputed or dropped. Categorical variables will need to be encoded.

In [5]:
df_train.describe()  # summary stats

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


The summary statistics reveal several data quality issues.
- We see negative values in columns that should be non-negative, such as -1.0 in Num_Bank_Accounts and -5.0 in Delay_from_due_date. These will need correction or removal.
- There are also extreme maximum values in Num_Bank_Accounts, Interest_Rate, and Num_Credit_Inquiries, suggesting outliers. We'll have to handle the outliers, either through clipping or filtering.
- The wide range of values across different columns confirms that feature scaling will be essential for our neural network.

In [6]:
df_train.describe(include='object')  # summary stats for non-numeric columns (unique values, top value, frequency)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Changed_Credit_Limit,Credit_Mix,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,100000,100000,100000,100000,100000,88592,92998,100000,100000,100000,90970,100000,95521,100000,98800,100000
unique,100000,12500,8,10139,1788,12501,16,18940,434,6260,749,4384,4,13178,404,3,91049,7,98792,3
top,0x1602,CUS_0xd40,January,Langep,38,#F%$D@*&8,_______,36585.12,3,Not Specified,19,_,Standard,1360.45,15 Years and 11 Months,Yes,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12500,44,2833,5572,7062,16,14386,1408,5327,2091,36479,24,446,52326,4305,25513,9,53174


Note:
- Many columns have high cardinality, which makes them difficult to encode directly for a baseline model.
- Columns like Payment_Behaviour and Type_of_Loan may be too granular or messy for a baseline model — we might drop them.
- Credit_History_Age is a messy text format that will need special parsing. It'll likely be converted to total months.
- The presence of placeholder values like _ in Amount_invested_monthly and _ in Credit_Mix indicates more data cleaning is required.

In [7]:
df_train["Credit_Score"].value_counts()  # counts of each category in the target column

Credit_Score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

The target variable has three categories: 'Standard', 'Poor', and 'Good'. 
- For our binary classification task, we will map 'Poor' to 1 (high risk) and both 'Standard' and 'Good' to 0 (low risk).
- The classes are somewhat imbalanced, with 'Poor' being the minority class, which is why using stratify during our data split is important.

In [8]:
df_train.isna().sum().sort_values(ascending=False)  # count missing values per column

Monthly_Inhand_Salary       15002
Type_of_Loan                11408
Name                         9985
Credit_History_Age           9030
Num_of_Delayed_Payment       7002
Amount_invested_monthly      4479
Num_Credit_Inquiries         1965
Monthly_Balance              1200
ID                              0
Changed_Credit_Limit            0
Payment_Behaviour               0
Total_EMI_per_month             0
Payment_of_Min_Amount           0
Credit_Utilization_Ratio        0
Outstanding_Debt                0
Credit_Mix                      0
Delay_from_due_date             0
Customer_ID                     0
Num_of_Loan                     0
Interest_Rate                   0
Num_Credit_Card                 0
Num_Bank_Accounts               0
Annual_Income                   0
Occupation                      0
SSN                             0
Age                             0
Month                           0
Credit_Score                    0
dtype: int64

The list confirms that Monthly_Inhand_Salary, Type_of_Loan, Name, Credit_History_Age, and Num_of_Delayed_Payment have a significant number of missing values.

In [9]:
df_train.duplicated().sum()  # count of fully duplicated rows

0

No duplicates.

# Data Cleaning and Feature Engineering

## Drop Irrelevant Columns

Some columns don't contribute to prediction, may introduce noise, or house personal information. We'll start by removing them.

In [10]:
# Create a list of columns to drop for clarity and reusability
columns_to_drop = [
    "Occupation",          # High cardinality, will simplify for baseline model
    "ID",                  # Unique identifier for the row, not a predictive feature
    "Customer_ID",         # Unique identifier for the customer, not useful for generalization
    "Month",               # Temporal label that adds noise without time-series modeling
    "Name",                # Personal identifier with high cardinality, not predictive
    "SSN",                 # Sensitive data, not useful for prediction
    "Type_of_Loan",        # Multi-valued string, too complex to encode for a baseline model
    "Payment_Behaviour"    # Descriptive strings that are hard to model without complex NLP
]

# Drop the specified columns from the DataFrame in-place
df_train.drop(columns=columns_to_drop, inplace=True)

In [11]:
df_train.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,23,19114.12,1824.843333,3,4,3,4,3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,312.49408867943663,Good
1,23,19114.12,NaN,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,284.62916249607184,Good
2,-500,19114.12,NaN,3,4,3,4,3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,331.2098628537912,Good
3,23,19114.12,NaN,3,4,3,4,5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,223.45130972736786,Good
4,23,19114.12,1824.843333,3,4,3,4,6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,341.48923103222177,Good


The unnecessary columns have been removed. The dataset is now leaner and more focused on the core financial attributes we want to model.

## Handling Credit_History_Age

The Credit_History_Age column is stored as text (for example, "22 Years and 1 Months"). To use it as a feature, we must convert it into a single numerical unit, like total months. 

In [12]:
# Define a function to convert the age string to total months
def convert_credit_age(age_str):
    # Check if the input is a string before trying to split it
    if isinstance(age_str, str):
        # Use a try-except block to handle potential parsing errors
        try:
            # Split the string into parts based on spaces
            parts = age_str.split()
            # Extract the years value and convert to integer
            years = int(parts[0])
            # Extract the months value and convert to integer
            months = int(parts[3])
            # Calculate the total number of months
            return (years * 12) + months
        # If any error occurs during parsing (e.g., unexpected format)
        except (ValueError, IndexError):
            # Return a NumPy Not-a-Number (NaN) for invalid formats
            return np.nan
    # If the input is not a string (e.g., already NaN), return NaN
    return np.nan

# Apply the conversion function to the 'Credit_History_Age' column
df_train['Credit_History_Age'] = df_train['Credit_History_Age'].apply(convert_credit_age)

The Credit_History_Age feature is now a numeric column representing the age in total months.

## Convert Object Type Numeric Columns

We'll convert columns that are stored as object but should be numeric. Using errors='coerce' will automatically turn any non-numeric text (like __ or other symbols) into NaN.

In [13]:
# List of columns that should be numeric but are stored as object
numeric_object_cols = [
    "Age",                        # age in years
    "Annual_Income",              # yearly income
    "Num_of_Loan",                # number of loans held
    "Num_of_Delayed_Payment",     # count of late payments
    "Changed_Credit_Limit",       # change in credit limit
    "Outstanding_Debt",           # total debt
    "Amount_invested_monthly",    # monthly investment amount
    "Monthly_Balance"             # leftover funds after expenses
]

# Loop through the list and convert each column to a numeric type
for col in numeric_object_cols:
    # pd.to_numeric converts values; errors='coerce' makes invalid entries into NaN
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')

## Clipping Outliers

For columns with illogical values (e.g., a negative age), we will clip them to a reasonable range.

In [14]:
# Clip the 'Age' column to a reasonable range (e.g., 18 to 100)
df_train['Age'] = df_train['Age'].clip(lower=18, upper=100)

# Clip 'Num_Bank_Accounts' to be non-negative
df_train['Num_Bank_Accounts'] = df_train['Num_Bank_Accounts'].clip(lower=0)

**Note on missing values**: I'm aware that the steps above, especially using errors='coerce', have created NaN (missing) values in the dataset. I'm intentionally leaving them in place for now. They will be handled systematically and safely in a later section using a scikit-learn imputation pipeline. This ensures that I don't introduce data leakage by imputing before we split our data.

# Encoding Target Variable

The target variable, Credit_Score, is categorical with values 'Good', 'Standard', and 'Poor'. For a binary classification model, we need to convert this into a numeric format. We will map the 'Poor' category to 1 (representing high risk) and the 'Standard' and 'Good' categories to 0 (representing low risk).

In [15]:
# Encode Credit_Score as binary classification target
# Map 'Poor' to 1 (high risk), and 'Standard'/'Good' to 0 (low risk)
df_train['Credit_Score'] = df_train['Credit_Score'].map(
    lambda x: 1 if x == 'Poor' else 0
)

# Confirm encoding
print("Encoded target values:", df_train['Credit_Score'].unique())  # should show [0, 1]

Encoded target values: [0 1]


# Data Splitting

## Feature Matrix and Target Vector

Before splitting, we'll separate the dataset into a feature matrix X and target variable y

In [16]:
# Separate features and target 
X = df_train.drop(columns=['Credit_Score'])  # all features
y = df_train['Credit_Score']                 # binary target

## Training and Validation Sets

In [17]:
# Split the data into training (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y,             # The features and target data to split
    test_size=0.2,    # Allocate 20% of the data to the validation set
    random_state=42,  # For reproducibility
    stratify=y        # Preserves proportion of target classes in both sets 
)

In [18]:
# Confirm the shapes of the splits
print("Training features shape:", X_train.shape)
print("Validation features shape:", X_val.shape)
print("Training target shape:", y_train.shape)
print("Validation target shape:", y_val.shape)
print("Training target distribution:\n", y_train.value_counts(normalize=True))
print("Validation target distribution:\n", y_val.value_counts(normalize=True))

Training features shape: (80000, 19)
Validation features shape: (20000, 19)
Training target shape: (80000,)
Validation target shape: (20000,)
Training target distribution:
 Credit_Score
0    0.710025
1    0.289975
Name: proportion, dtype: float64
Validation target distribution:
 Credit_Score
0    0.71
1    0.29
Name: proportion, dtype: float64


# Preprocessing Pipeline

Using scikit-learn's Pipeline and ColumnTransformer, we will create a single object that handles all imputation, encoding, and scaling. This approach is clean, less prone to errors, and prevents data leakage by fitting only on the training data.

## Identify Feature Types

First, we need to create separate lists of our numeric and categorical column names. The ColumnTransformer will use these lists to know which pipeline to apply to which columns.

In [19]:
# Create a list of column names with numeric data types
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create a list of column names with the object data type
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Print the lists to verify that our features have been correctly categorized
print("Numeric features identified:", numeric_features)
print("Categorical features identified:", categorical_features)

Numeric features identified: ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']
Categorical features identified: ['Credit_Mix', 'Payment_of_Min_Amount']


Our feature names are now organized into two lists, ready to be passed to our transformer.

## Numeric and Categorical Pipelines

Next, we define the series of steps for each data type. We'll create one pipeline for numeric data (impute then scale) and another for categorical data (impute then one-hot encode).

In [20]:
# Define the transformation pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    # Step 1: Impute missing values, replacing NaNs with the median of the column
    ('imputer', SimpleImputer(strategy='median')),
    # Step 2: Scale the features to have a mean of 0 and a standard deviation of 1
    ('scaler', StandardScaler())
])

In [21]:
# Define the transformation pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    # Step 1: Impute missing values, replacing NaNs with the most frequent value (mode)
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # Step 2: Convert categorical features into one-hot encoded columns
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' prevents errors if validation data has a category not in training data
])

We have now defined two distinct mini-pipelines.

## Combine Pipelines

Now, we bring everything together. The ColumnTransformer applies the correct transformation pipeline to the correct columns.

In [22]:
# Create the main preprocessor object by combining the transformers
preprocessor = ColumnTransformer(
    transformers=[
        # Apply the numeric_transformer to all columns in the numeric_features list
        ('num', numeric_transformer, numeric_features),
        # Apply the categorical_transformer to all columns in the categorical_features list
        ('cat', categorical_transformer, categorical_features)
    ],
    # Keep other columns (if any) instead of dropping them. We have none, but this is good practice.
    remainder='passthrough'
)

## Apply the Preprocessor

Finally, let's use our preprocessor. We will fit and transform it on the training data (X_train) and then only transform the validation data (X_val). This is the crucial step that prevents data leakage; the imputation and scaling parameters are learned only from the training data.

In [23]:
# Fit the preprocessor on the training data and transform X_train
X_train_processed = preprocessor.fit_transform(X_train)

# Use the fitted preprocessor to transform the validation data
X_val_processed = preprocessor.transform(X_val)

# Print the shapes of the processed data to see the final result
print("Processed training data shape:", X_train_processed.shape)
print("Processed validation data shape:", X_val_processed.shape)

Processed training data shape: (80000, 24)
Processed validation data shape: (20000, 24)


Our data is now properly prepared. The final step is to save these processed datasets and the preprocessor object itself.

# Exporting Preprocessed Data & Preprocessor

The final step in this notebook is to save our outputs so they can be easily loaded into the next notebook for model training. We will save the processed datasets (X_train, X_val) and the target labels (y_train, y_val). Most importantly, we will save the fitted preprocessor object itself.

## Saving Preprocessed Datasets

While our preprocessor outputs NumPy arrays, it's best practice to convert them back to pandas DataFrames with meaningful column names before saving. This is especially important for this project, as it will make interpreting our model with SHAP much easier later.

In [24]:
# Get the feature names from the one-hot encoding step
ohe_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)

# Combine the original numeric feature names with the new one-hot encoded feature names
final_feature_names = numeric_features + ohe_feature_names.tolist()

# Convert the processed training data array back to a DataFrame with correct column names
X_train_df = pd.DataFrame(X_train_processed, columns=final_feature_names, index=X_train.index)

# Convert the processed validation data array back to a DataFrame with correct column names
X_val_df = pd.DataFrame(X_val_processed, columns=final_feature_names, index=X_val.index)

In [25]:
# Save the processed training features DataFrame to a CSV file
X_train_df.to_csv("../data/X_train_processed.csv", index=False)

# Save the processed validation features DataFrame to a CSV file
X_val_df.to_csv("../data/X_val_processed.csv", index=False)

# Save the training target Series to a CSV file
y_train.to_csv("../data/y_train.csv", index=False, header=True)

# Save the validation target Series to a CSV file
y_val.to_csv("../data/y_val.csv", index=False, header=True)

Our fully processed training and validation datasets are now saved as CSV files. The next notebook can load these directly to begin model training without needing to re-run any of these preprocessing steps.

## Saving Preprocessing Pipeline

We save the preprocessor object, which has been fitted on our training data. This single file contains all the necessary information (medians for imputation, scales, one-hot encoding mappings) to process new, unseen data—like the official test.csv—in exactly the same way.

In [26]:
# Define the file path for saving the preprocessor object (in a 'models' folder)
preprocessor_path = "../models/preprocessor.pkl"

# Use joblib to serialize and save the fitted preprocessor object to a binary file
joblib.dump(preprocessor, preprocessor_path)

['../models/preprocessor.pkl']

Our preprocessor is now saved. This concludes the data preprocessing notebook. We have successfully cleaned the data, engineered features, and created a robust, reusable pipeline to prepare the data for modeling.

In [27]:
# Get working directory to see where all this is saved
print(os.getcwd())

C:\Users\barbe\OneDrive\URI DS Program\566 Advanced Topics in Machine Learning\Project\neural-credit-risk-explainer\notebooks
